# Análisis de series temporales

## Introducción

### Propósito del proyecto

<!-- TODO: Explicar el propósito del proyecto -->

### Descripción de los conjuntos de datos utilizados

<!-- TODO: Explicar conjuntos de datos utilizados -->

### Operaciones de transformación de datos

<!-- TODO: Explicar dataframe de datos únicos -->

<!-- TODO: Explicar dataframe multiindice para panel -->

<!-- TODO: Explicar dataframe para data streams -->

### Ajustes de granularidad

<!-- TODO: Explicar que se reducira de dias a semanas o grupos de 2 semanas -->

## Análisis exploratorio de datos

### Visualización de datos

<!-- TODO: Visualizar shit -->

### Tendencia y estacionalidad

<!-- TODO: Describir tendencia si la hubiera -->

<!-- TODO: Describir estacionalidad si la hubiera -->

### Estacionariedad

<!-- TODO: Explicar qué es -->

<!-- TODO: Comprobar agdfuller sin diferenciar -->

<!-- TODO: Comprobar agdfuller diferenciando -->

### Autocorrelación (_ACF_) y autocorrelación parcial (_PACF_)

<!-- TODO: Explicar por encima correlación y autocorrelación -->

<!-- TODO: Realizar graficos -->

<!-- *: Explicar valores de ACF y PACF que importan para ARIMA -->

## Uso de series temporales

### Series univariadas

### Series multivariadas

### Series de tipo *panel*

## Uso de flujos de datos (_Data Streams_)